# Ambiente de Testes

In [ ]:
import pandas as pd
import pyodbc
from flask_bcrypt import generate_password_hash

In [ ]:
SERVER = r'.\SQLEXPRESS'
DRIVER = 'ODBC Driver 17 for SQL Server'
CONNECTIONSTRING = f'DRIVER={DRIVER};SERVER={SERVER};Trusted_Connection=yes;'
CONN = pyodbc.connect(CONNECTIONSTRING, autocommit = True)
CURSOR = CONN.cursor()

In [ ]:
CURSOR.execute(''' DROP DATABASE IF EXISTS dbOneplay ''')
CURSOR.execute(''' CREATE DATABASE dbOneplay ''')

In [ ]:
CURSOR.execute(''' USE dbOneplay ''')

In [ ]:
TABELAS = {}
TABELAS['tblJogos'] = ('''
    create table tblJogos(
	JogoID int not null identity(1,1) primary key,
	CategoriaID int,--FK
	ExibirHome bit default 0,
	Ordem int,
	Nome varchar(max),
	Frase varchar(max),
	Descricao varchar(max),
	Download varchar(max),
	VersaoDados datetime null,
    );
''')
TABELAS['TRIGGER_tblJogos1'] = ('''
	CREATE TRIGGER Valor_Padrao_tblJogos
    ON tblJogos
    AFTER INSERT
    AS
    BEGIN
	UPDATE tblJogos SET Nome = 'Nome Jogo ID '+CAST(i.JogoID AS VARCHAR(MAX)) FROM tblJogos t INNER JOIN inserted i ON t.JogoID = i.JogoID WHERE t.Nome IS NULL;
	UPDATE tblJogos SET Frase = 'Frase Jogo ID '+CAST(i.JogoID AS VARCHAR(MAX)) FROM tblJogos t INNER JOIN inserted i ON t.JogoID = i.JogoID WHERE t.Frase IS NULL;
	UPDATE tblJogos SET Descricao = 'Descricao Jogo ID '+CAST(i.JogoID AS VARCHAR(MAX)) FROM tblJogos t INNER JOIN inserted i ON t.JogoID = i.JogoID WHERE t.Descricao IS NULL;
    END;
''')
TABELAS['TRIGGER_tblJogos2'] = ('''
	CREATE TRIGGER Versao_Dados_tblJogos
	ON tblJogos
	AFTER UPDATE,INSERT
	AS
	BEGIN
		UPDATE tblJogos SET VersaoDados = GETDATE() WHERE JogoID IN (SELECT DISTINCT JogoID FROM inserted)
	END;
''')

TABELAS['tblCategorias'] = ('''
    create table tblCategorias(
	CategoriaID int not null identity(1,1) primary key,
	Categoria varchar(max) not null,
	VersaoDados datetime null,
    );
''')
TABELAS['TRIGGER_tblCategorias'] = ('''
	CREATE TRIGGER Versao_Dados_tblCategorias
	ON tblCategorias
	AFTER UPDATE,INSERT
	AS
	BEGIN
		UPDATE tblCategorias SET VersaoDados = GETDATE() WHERE CategoriaID IN (SELECT DISTINCT CategoriaID FROM inserted)
	END;
''')

TABELAS['tblImagens'] = ('''
    create table tblImagens(
	ImagensID int not null identity(1,1) primary key,
	JogoID int not null,--FK
	Titulo1 varchar(max),
	Titulo2 varchar(max),
	Descricao1 varchar(max),
	Descricao2 varchar(max),
	Imagem1 VARBINARY(max),
	Imagem2 VARBINARY(max),
	VersaoDados datetime null,
    );
''')
TABELAS['TRIGGER_tblImagens'] = ('''
	CREATE TRIGGER Versao_Dados_tblImagens
	ON tblImagens
	AFTER UPDATE,INSERT
	AS
	BEGIN
		UPDATE tblImagens SET VersaoDados = GETDATE() WHERE ImagensID IN (SELECT DISTINCT ImagensID FROM inserted)
	END;
''')

TABELAS['tblBanners'] = ('''
    create table tblBanners(
	BannerID int not null identity(1,1) primary key,
	JogoID int not null,--FK
	Home VARBINARY(max),
	Banner VARBINARY(max),
	VersaoDados datetime null,
    );                      
''')
TABELAS['TRIGGER_tblBanners'] = ('''
	CREATE TRIGGER Versao_Dados_tblBanners
	ON tblBanners
	AFTER UPDATE,INSERT
	AS
	BEGIN
		UPDATE tblBanners SET VersaoDados = GETDATE() WHERE BannerID IN (SELECT DISTINCT BannerID FROM inserted)
	END;
''')

TABELAS['tblVideos'] = ('''
    create table tblVideos(
	VideoID int not null identity(1,1) primary key,
	JogoID int not null,--FK
	Titulo varchar(max),
	Descricao varchar(max),
	Link varchar(max),
	VersaoDados datetime null,
    );                  
''')
TABELAS['TRIGGER_tblVideos'] = ('''
	CREATE TRIGGER Versao_Dados_tblVideos
	ON tblVideos
	AFTER UPDATE,INSERT
	AS
	BEGIN
		UPDATE tblVideos SET VersaoDados = GETDATE() WHERE VideoID IN (SELECT DISTINCT VideoID FROM inserted)
	END;
''')

TABELAS['tblTutoriais'] = ('''
    create table tblTutoriais(
	TutorialID int not null identity(1,1) primary key,
	JogoID int not null,--FK
	Titulo varchar(max),
	Descricao varchar(max),
	Passo varchar(max),
	DescricaoPasso varchar(max),
	VersaoDados datetime null,
    );                   
''')
TABELAS['TRIGGER_tblTutoriais'] = ('''
	CREATE TRIGGER Versao_Dados_tblTutoriais
	ON tblTutoriais
	AFTER UPDATE,INSERT
	AS
	BEGIN
		UPDATE tblTutoriais SET VersaoDados = GETDATE() WHERE TutorialID IN (SELECT DISTINCT TutorialID FROM inserted)
	END;
''')

TABELAS['tblUsuarios'] = ('''
    CREATE TABLE tblUsuarios(
	UsuarioID INT NOT NULL identity(1,1) primary key,
	Nome VARCHAR(MAX) NOT NULL,
	Senha VARCHAR(MAX) NOT NULL,
	VersaoDados datetime null,
    );                    
''')
TABELAS['TRIGGER_tblUsuarios'] = ('''
	CREATE TRIGGER Versao_Dados_tblUsuarios
	ON tblUsuarios
	AFTER UPDATE,INSERT
	AS
	BEGIN
		UPDATE tblUsuarios SET VersaoDados = GETDATE() WHERE UsuarioID IN (SELECT DISTINCT UsuarioID FROM inserted)
	END;
''')

TABELAS['ForeignKeys'] = ('''
    ALTER TABLE tblBanners ADD CONSTRAINT FK_tblBanners_tblJogos_JogoID FOREIGN KEY(JogoID) REFERENCES tblJogos(JogoID)
    ALTER TABLE tblImagens ADD CONSTRAINT FK_tblImagens_tblJogos_JogoID FOREIGN KEY(JogoID) REFERENCES tblJogos(JogoID)
    ALTER TABLE tblTutoriais ADD CONSTRAINT FK_tblTutoriais_tblJogos_JogoID FOREIGN KEY(JogoID) REFERENCES tblJogos(JogoID)
    ALTER TABLE tblVideos ADD CONSTRAINT FK_tblVideos_tblJogos_JogoID FOREIGN KEY(JogoID) REFERENCES tblJogos(JogoID)
    ALTER TABLE tblJogos ADD CONSTRAINT FK_tblJogos_tblCategorias_CategoriaID FOREIGN KEY(CategoriaID) REFERENCES tblCategorias(CategoriaID)
''')

for tabela_nome in TABELAS:
    tabela_sql = TABELAS[tabela_nome]
    try:
        print('Criando {}:'.format(tabela_nome))
        CURSOR.execute(tabela_sql)
    except pyodbc.Error as ex:
        sqlstate = ex.args[0] 
        if sqlstate == '42000':
            print("Erro de sintaxe SQL.")
        else:
            print(f"Ocorreu um erro: {ex}")
        exit()

In [ ]:
planilhas = pd.ExcelFile('Jogos.xlsx')
nomes_planilhas = planilhas.sheet_names
dados = {sheet: planilhas.parse(sheet) for sheet in nomes_planilhas}
dados['tblJogos']

In [ ]:
dados['IMG']

In [ ]:
dados['tblJogos'].iloc[0].iloc[3]

In [ ]:
print(len(dados['tblJogos']))
print(len(dados['IMG'].iloc[0]))
print(len(dados['IMG'].iloc[0].iloc[3]))

In [ ]:
CURSOR.executemany('insert into tblCategorias(Categoria) values(?)',dados['tblCategorias'].values.tolist())

In [ ]:
CURSOR.executemany('insert into tblJogos(CategoriaID,ExibirHome,Ordem,Nome,Frase,Descricao,Download) values(?,?,?,?,?,?,?)',dados['tblJogos'].values.tolist())
#CURSOR.execute('INSERT INTO tblJogos DEFAULT VALUES')

In [ ]:
CURSOR.executemany('insert into tblImagens(JogoID,Titulo1,Titulo2,Descricao1,Descricao2) values(?,?,?,?,?)',dados['tblImagens'].values.tolist())

In [ ]:
CURSOR.executemany('insert into tblBanners(JogoID) values(?)',dados['tblBanners'].values.tolist())

In [ ]:
CURSOR.executemany('insert into tblVideos(JogoID,Titulo,Descricao,Link) values(?,?,?,?)',dados['tblVideos'].values.tolist())

In [ ]:
CURSOR.executemany('insert into tblTutoriais(JogoID,Titulo,Descricao,Passo,DescricaoPasso) values(?,?,?,?,?)',dados['tblTutoriais'].values.tolist())

In [ ]:
CURSOR.executemany('insert into tblUsuarios(Nome,Senha) values(?,?)',dados['tblUsuarios'].values.tolist())

In [ ]:
CURSOR.execute('''select a.JogoID,a.Nome,a.Descricao,b.Banner from tblJogos a inner join tblBanners b on b.JogoID = a.JogoID''').fetchone()

In [ ]:
r = CURSOR.execute('select * from tblTutoriais where JogoID = ?',1).fetchall()
r[0]

In [ ]:
CURSOR.close()
CONN.close()